In [10]:
import torch
import torchvision
import psycopg2
import torchvision.transforms as transforms
import torch.nn as nn
from tqdm.auto import tqdm
import io
from PIL import Image
from sklearn.metrics import roc_curve, auc
from torch.utils.data import Dataset
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pickle


In [11]:
torch.cuda.empty_cache()
batch_size=32
num_classes=23
epochs=15
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [12]:
transformation = transforms.Compose(
    [    
        transforms.Resize((256, 256)),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

base_transform = torchvision.transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

In [13]:

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
    def __getitem__(self, index):
        img = self.data[index]
        if self.transform:
            img = self.transform(img)
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.data)

In [14]:

conn_select = psycopg2.connect(
    database="hse_medical",
    user='hse_medical',
    password='123456',
    host='127.0.0.1',
    port='5450',
    options="-c search_path=analyze_medical"
)

conn_select.autocommit = True

transform = transforms.ToTensor()

def get_connection():
    return conn_select

In [15]:


cursor = conn_select.cursor()

sql1 = f'''select 
    target,
    image from medical_pictures_train;'''
cursor.execute(sql1)
data_postgres = cursor.fetchall()
print(f"datatrain_size : {len(data_postgres)}")
cursor.close()
targets = []
images = []
for data in data_postgres:
    targets.append(data[0])
    image = Image.open(io.BytesIO(data[1]))
    images.append(image)
    

data_train = CustomDataset(images, targets,transform=transformation)

datatrain_size : 15588


In [16]:
cursor_test= conn_select.cursor()

sql2 = f'''select 
    target,
    image from medical_pictures_test;'''
cursor_test.execute(sql2)
data_postgres_test = cursor_test.fetchall()
cursor_test.close()
targets_test = []
images_test = []


for data in data_postgres_test:
    targets_test.append(data[0])
    bytes_io = io.BytesIO(data[1])
    image_open = Image.open(bytes_io)
    images_test.append(image_open)
    
    

data_test = CustomDataset(images_test, targets_test,transform=base_transform)

In [17]:
train_dataloader = torch.utils.data.DataLoader(
    data_train, batch_size=batch_size, shuffle=True, num_workers=4
)
val_dataloader = torch.utils.data.DataLoader(
    data_test, batch_size=batch_size, shuffle=False, num_workers=4
)

In [18]:

model = torchvision.models.resnet18(pretrained=True)
model.to(device)

# Modify the last layer for the number of classes in your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


/home/roman/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/roman/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/roman/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.5MB/s]


In [19]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_dataloader), 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:  
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
    model.eval()
    train_preds = []
    train_labels = []
    with torch.no_grad():
        for images, labels in tqdm(train_dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
    train_accuracy = accuracy_score(train_labels, train_preds)
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}")
    
print('Finished Training')
with open('resnet18.pickle', 'wb') as f:
    pickle.dump(model, f)

  0%|          | 0/488 [00:00<?, ?it/s]

[1,   100] loss: 2.636
[1,   200] loss: 2.262
[1,   300] loss: 2.114
[1,   400] loss: 2.016


  0%|          | 0/488 [00:00<?, ?it/s]

Epoch [1/15], Loss: 176.8620, Train Accuracy: 0.5004


  0%|          | 0/488 [00:00<?, ?it/s]

...

Epoch [14/15], Loss: 24.0272, Train Accuracy: 0.8947


  0%|          | 0/488 [00:00<?, ?it/s]

[15,   100] loss: 0.308
[15,   200] loss: 0.225
[15,   300] loss: 0.217
[15,   400] loss: 0.257


  0%|          | 0/488 [00:00<?, ?it/s]

Epoch [15/15], Loss: 25.3460, Train Accuracy: 0.9298
Finished Training


In [25]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score

correct = 0
total = 0
y_true = []
y_pred = []
predictions = []
with torch.no_grad():
    for data in tqdm(val_dataloader):
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(probabilities.cpu().numpy())
        
        
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
test_roc_auc = roc_auc_score(y_true, y_pred,multi_class='ovr')
test_classification_report = classification_report(y_true, predictions)
precision = precision_score(y_true,predictions,average='macro')
test_accuracy = accuracy_score(y_true, predictions)
test_recall_macro = recall_score(y_true, predictions,average='macro')
test_recall_weighted = recall_score(y_true, predictions,average='weighted')
test_recall_micro = recall_score(y_true, predictions,average='micro')

print(f"Test Accuracy: {test_accuracy:.4f}, "
      f"Test Recall micro: {test_recall_micro:}",
      f"Test Recall macro: {test_recall_macro:}",
      f"Test Recall weighted: {test_recall_weighted:}",
      f"test precision {precision:.4f} " 
      f"Test ROC-AUC: {test_roc_auc:.4f}")

print(f"Classification Report:\n{test_classification_report}")


  0%|          | 0/126 [00:00<?, ?it/s]

Accuracy of the network on the test images: 57 %
Test Accuracy: 0.5749, Test Recall micro: 0.5749007936507936 Test Recall macro: 0.5041740842462662 Test Recall weighted: 0.5749007936507936 test precision 0.5684 Test ROC-AUC: 0.9250
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.54      0.57       314
           1       0.88      0.89      0.89       317
           2       0.60      0.59      0.59       293
           3       0.51      0.61      0.56       357
           4       0.60      0.62      0.61       330
           5       0.34      0.38      0.36       110
           6       0.46      0.38      0.42       143
           7       0.77      0.50      0.61       116
           8       0.80      0.72      0.76       261
           9       0.58      0.49      0.53       123
          10       0.50      0.48      0.49       113
          11       0.67      0.14      0.23        73
          12       0.45      0.46      0.4

AxisError: axis 1 is out of bounds for array of dimension 1